<a href="https://colab.research.google.com/github/P3aar/Cartorioebac/blob/main/AgregacoesAvancadasP1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
spark = SparkSession.builder.getOrCreate()

In [5]:
produtos = spark.read.csv('drive/MyDrive/Planilha excel ANALISTA DE DADOS/produtos.csv', header=True, inferSchema=True)
vendedores = spark.read.csv('drive/MyDrive/Planilha excel ANALISTA DE DADOS/vendedores.csv', header=True, inferSchema=True)
clientes = spark.read.csv('drive/MyDrive/Planilha excel ANALISTA DE DADOS/clientes.csv', header=True, inferSchema=True)
itens_pedido = spark.read.csv('drive/MyDrive/Planilha excel ANALISTA DE DADOS/itens_pedido.csv', header=True, inferSchema=True)
pagamentos_pedido = spark.read.csv('drive/MyDrive/Planilha excel ANALISTA DE DADOS/pagamentos_pedido.csv', header=True, inferSchema=True)
avaliacoes_pedido = spark.read.csv('drive/MyDrive/Planilha excel ANALISTA DE DADOS/avaliacoes_pedido.csv', header=True, inferSchema=True)
pedidos = spark.read.csv('drive/MyDrive/Planilha excel ANALISTA DE DADOS/pedidos.csv', header=True, inferSchema=True)

In [6]:
pedidos.show(2)

+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+
|           id_pedido|          id_cliente|status_pedido| data_compra_pedido|data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|
+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|    delivered|2017-10-02 10:56:33|  2017-10-02 11:07:15|      2017-10-04 19:55:00| 2017-10-10 21:25:13|  2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|    delivered|2018-07-24 20:41:37|  2018-07-26 03:24:27|      2018-07-26 14:31:00| 2018-08-07 15:27:45|  2018-08-13 00:00:00|
+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------

In [7]:
pedidos.withColumn('compra_anual', year('data_compra_pedido')).withColumn('compra_mensal', month('data_compra_pedido')).groupBy('compra_anual', 'compra_mensal').count().orderBy(desc('compra_anual'), 'compra_mensal').show()

+------------+-------------+-----+
|compra_anual|compra_mensal|count|
+------------+-------------+-----+
|        2018|            1| 7269|
|        2018|            2| 6728|
|        2018|            3| 7211|
|        2018|            4| 6939|
|        2018|            5| 6873|
|        2018|            6| 6167|
|        2018|            7| 6292|
|        2018|            8| 6512|
|        2018|            9|   16|
|        2018|           10|    4|
|        2017|            1|  800|
|        2017|            2| 1780|
|        2017|            3| 2682|
|        2017|            4| 2404|
|        2017|            5| 3700|
|        2017|            6| 3245|
|        2017|            7| 4026|
|        2017|            8| 4331|
|        2017|            9| 4285|
|        2017|           10| 4631|
+------------+-------------+-----+
only showing top 20 rows



In [8]:
avaliacoes_pedido.show(2)
itens_pedido.show(2)
avaliacoes_pedido.join(itens_pedido, 'id_pedido').groupBy('id_produto').agg(format_number(avg('nota_avaliacao'), 2).alias('media_avaliacao')).show()

+--------------------+--------------------+--------------+-----------------+-------------------+----------------------+-----------------------+
|        id_avaliacao|           id_pedido|nota_avaliacao|titulo_comentario|mensagem_comentario|data_criacao_avaliacao|data_resposta_avaliacao|
+--------------------+--------------------+--------------+-----------------+-------------------+----------------------+-----------------------+
|7bc2406110b926393...|73fc7af87114b3971...|             4|             NULL|               NULL|   2018-01-18 00:00:00|    2018-01-18 21:46:59|
|80e641a11e56f04c1...|a548910a1c6147796...|             5|             NULL|               NULL|   2018-03-10 00:00:00|    2018-03-11 03:05:13|
+--------------------+--------------------+--------------+-----------------+-------------------+----------------------+-----------------------+
only showing top 2 rows

+--------------------+--------------+--------------------+--------------------+-------------------+-----+------

In [9]:
# converter timestamp para long (segundos)
pedidos_long = pedidos.withColumn('data_compra_pedido', col('data_compra_pedido').cast('long'))
pedidos_long = pedidos_long.withColumn('data_aprovacao_pedido', col('data_aprovacao_pedido').cast('long'))

# converter em horas
pedidos_horas = pedidos_long.withColumn('diferenca_horas',
                              (col('data_aprovacao_pedido') - col('data_compra_pedido')) / 3600)

pedidos_horas.select('data_compra_pedido', 'data_aprovacao_pedido','diferenca_horas').show()

+------------------+---------------------+-------------------+
|data_compra_pedido|data_aprovacao_pedido|    diferenca_horas|
+------------------+---------------------+-------------------+
|        1506941793|           1506942435|0.17833333333333334|
|        1532464897|           1532575467|  30.71388888888889|
|        1533717529|           1533718523| 0.2761111111111111|
|        1511033286|           1511034359| 0.2980555555555556|
|        1518556719|           1518560429| 1.0305555555555554|
|        1499637425|           1499638213|0.21888888888888888|
|        1491913328|           1492089917|            49.0525|
|        1494940230|           1494940931| 0.1947222222222222|
|        1485196149|           1485312647|  32.36055555555556|
|        1501329302|           1501329932|              0.175|
|        1494963670|           1494964218|0.15222222222222223|
|        1499975891|           1499976608|0.19916666666666666|
|        1528365979|           1528513992|  41.11472222

In [12]:
pedidos.withColumn(
    'tempo_aprovacao',
    col('data_aprovacao_pedido').cast('long') - col('data_compra_pedido').cast('long')
).groupBy().agg(
    (avg('tempo_aprovacao') / 3600).alias('tempo_medio_aprovacao_horas')
).show()

+---------------------------+
|tempo_medio_aprovacao_horas|
+---------------------------+
|         10.419094301919692|
+---------------------------+



In [14]:
pedidos.select('data_entrega_cliente', 'data_aprovacao_pedido').show(2)

pedidos_antecipados = pedidos.withColumn(
    'entregue_antecipadas',
    when(col('data_entrega_cliente') < col('data_estimada_entrega'), 1).otherwise(0)
).groupBy().agg(
    count(when(col('entregue_antecipadas') == 1, True)).alias('total_entregue_antecipadas')
)

pedidos_antecipados.show()

+--------------------+---------------------+
|data_entrega_cliente|data_aprovacao_pedido|
+--------------------+---------------------+
| 2017-10-10 21:25:13|  2017-10-02 11:07:15|
| 2018-08-07 15:27:45|  2018-07-26 03:24:27|
+--------------------+---------------------+
only showing top 2 rows

+--------------------------+
|total_entregue_antecipadas|
+--------------------------+
|                     88649|
+--------------------------+



In [15]:
pedidos.select('data_entrega_cliente', 'data_aprovacao_pedido').show(2)

tempo_medio_envio_status = pedidos.withColumn(
    'tempo_envio',
    col('data_entrega_cliente').cast('long') - col('data_compra_pedido').cast('long')
).groupBy('status_pedido').agg(
    format_number((avg('tempo_envio') / 86400),2).alias('tempo_medio_envio_dias')
)

tempo_medio_envio_status.show()

+--------------------+---------------------+
|data_entrega_cliente|data_aprovacao_pedido|
+--------------------+---------------------+
| 2017-10-10 21:25:13|  2017-10-02 11:07:15|
| 2018-08-07 15:27:45|  2018-07-26 03:24:27|
+--------------------+---------------------+
only showing top 2 rows

+-------------+----------------------+
|status_pedido|tempo_medio_envio_dias|
+-------------+----------------------+
|      shipped|                  NULL|
|     canceled|                 20.36|
|     approved|                  NULL|
|     invoiced|                  NULL|
|      created|                  NULL|
|    delivered|                 12.56|
|  unavailable|                  NULL|
|   processing|                  NULL|
+-------------+----------------------+



In [16]:
pagamentos_pedido.show(2)
itens_pedido.show(2)
pedidos.show(2)
clientes.show()

+--------------------+-------------------+--------------+------------------+---------------+
|           id_pedido|sequencia_pagamento|tipo_pagamento|parcelas_pagamento|valor_pagamento|
+--------------------+-------------------+--------------+------------------+---------------+
|b81ef226f3fe1789b...|                  1|   credit_card|                 8|          99.33|
|a9810da82917af2d9...|                  1|   credit_card|                 1|          24.39|
+--------------------+-------------------+--------------+------------------+---------------+
only showing top 2 rows

+--------------------+--------------+--------------------+--------------------+-------------------+-----+-----------+
|           id_pedido|item_id_pedido|          id_produto|         id_vendedor|  data_limite_envio|preco|valor_frete|
+--------------------+--------------+--------------------+--------------------+-------------------+-----+-----------+
|00010242fe8c5a6d1...|             1|4244733e06e7ecb49...|48436

In [17]:
vendas_pedido_join = pagamentos_pedido.join(itens_pedido, 'id_pedido').join(pedidos,'id_pedido').join(clientes, 'id_cliente')

vendas_por_estado = vendas_pedido_join.groupBy('estado_cliente').agg(sum('valor_pagamento').alias('total_vendas'))

vendas_por_estado.show()

+--------------+------------------+
|estado_cliente|      total_vendas|
+--------------+------------------+
|            SC| 786343.7099999998|
|            RO|           65886.0|
|            PI|136779.95999999996|
|            AM|34753.299999999996|
|            RR|          12462.21|
|            GO|513879.00000000035|
|            TO|          72281.17|
|            MT|256804.61999999994|
|            SP| 7597209.659999931|
|            ES| 405805.3400000009|
|            PB| 180984.1899999999|
|            RS|1147277.0000000023|
|            MS|164337.27999999997|
|            AL|         111284.42|
|            MG|2326151.6399999885|
|            PA|261788.34999999992|
|            BA| 797410.3600000021|
|            SE| 88437.50999999998|
|            PE|376377.26999999967|
|            CE| 343847.8300000001|
+--------------+------------------+
only showing top 20 rows



In [19]:
vendas_por_estado_tipo_pagamento = vendas_pedido_join.groupBy('estado_cliente', 'tipo_pagamento') \
    .agg(sum('valor_pagamento').alias('total_vendas')) \
    .orderBy(col('estado_cliente'),desc('total_vendas'))

vendas_por_estado_tipo_pagamento = vendas_por_estado_tipo_pagamento.withColumn('total_vendas', format_number(col('total_vendas'), 2))

vendas_por_estado_tipo_pagamento.show()

+--------------+--------------+------------+
|estado_cliente|tipo_pagamento|total_vendas|
+--------------+--------------+------------+
|            AC|   credit_card|   19,929.65|
|            AC|        boleto|    3,908.46|
|            AC|       voucher|      797.13|
|            AC|    debit_card|      349.62|
|            AL|   credit_card|   91,964.27|
|            AL|        boleto|   16,388.24|
|            AL|       voucher|    1,705.79|
|            AL|    debit_card|    1,226.12|
|            AM|   credit_card|   26,751.24|
|            AM|        boleto|    7,639.22|
|            AM|       voucher|      210.82|
|            AM|    debit_card|      152.02|
|            AP|   credit_card|   16,471.08|
|            AP|        boleto|    4,934.20|
|            AP|       voucher|      237.42|
|            BA|   credit_card|  641,001.97|
|            BA|        boleto|  132,269.38|
|            BA|       voucher|   17,106.15|
|            BA|    debit_card|    7,032.86|
|         

In [25]:
produtos_categoria_nulos = produtos.groupBy('categoria_produto').agg(
    count(when(col('tamanho_descricao_produto').isNull(), True)).alias('qtd_produtos_nulos')
)
produtos_categoria_nulos.show()

produtos_categoria_nao_nulos = produtos.groupBy('categoria_produto').agg(
    count(when(col('tamanho_descricao_produto').isNotNull(), True)).alias('qtd_produtos_naoNulos')
)
produtos_categoria_nao_nulos.show()

+--------------------+------------------+
|   categoria_produto|qtd_produtos_nulos|
+--------------------+------------------+
|                 pcs|                 0|
|               bebes|                 0|
|               artes|                 0|
|           cine_foto|                 0|
|    moveis_decoracao|                 0|
|            pc_gamer|                 0|
|construcao_ferram...|                 0|
|tablets_impressao...|                 0|
|fashion_roupa_mas...|                 0|
|   artigos_de_festas|                 0|
|    artigos_de_natal|                 0|
|          la_cuisine|                 0|
|              flores|                 0|
|     livros_tecnicos|                 0|
|                NULL|               610|
|      telefonia_fixa|                 0|
|construcao_ferram...|                 0|
|          cool_stuff|                 0|
|    eletrodomesticos|                 0|
|   livros_importados|                 0|
+--------------------+------------

In [26]:
spark.stop()